# Make histogramms for any quantities

- author Sylvie Dagoret-Campagne
- creation date October 18th 2018
- update December 2022

In [ ]:
# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib
import matplotlib as mpl
matplotlib.rc_file("templates/matplotlibrc")
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%matplotlib inline

In [ ]:
import os
import re
import numpy as np
#from mpl_toolkits.basemap import Basemap
from matplotlib import colors
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

In [ ]:
import datetime as dt

In [ ]:
YEARSTR="2022"
YEARNUM=int(YEARSTR)

In [ ]:
#VAR_name='TOTANGSTR'
#VAR_title='Angstrom Parameter'
#VAR_unit='@470-870nm'
#VAR_min=0
#VAR_max=2

In [ ]:
VAR_name='TOTEXTTAU'
VAR_title='Extinction'
VAR_unit='@550nm'
VAR_min=0
VAR_max=0.5

In [ ]:
obs="lsst"

In [ ]:
yearfile="MERRA2_"+YEARSTR+"_tavg1_2d_aer_Nx_M2T1NXAER_" + obs + "_AllYear.csv"

In [ ]:
yearfile

In [ ]:
df_year=pd.read_csv(yearfile)

In [ ]:
df_year.head()

## Per year

In [ ]:
df_year.describe()

In [ ]:
df_year.quantile([0.05,0.1,0.5,0.9,0.95])

In [ ]:
var_year=df_year[VAR_name]

In [ ]:
var_year.replace(to_replace=0, value=np.nan)

In [ ]:
mean_var_y = np.mean(var_year)
variance_var_y = np.var(var_year)
sigma_var_y = np.sqrt(variance_var_y)

In [ ]:
mean_var_y=np.trunc(mean_var_y*1000)/1000
sigma_var_y=np.trunc(sigma_var_y*1000)/1000

In [ ]:
title = "Yearly variation of {} (year {}), mean = {} std = {}".format(VAR_title,YEARSTR,mean_var_y ,sigma_var_y)

In [ ]:
plt.figure(figsize=(14,8))
n, bins, patches = plt.hist(var_year, 50, density=False, facecolor='blue', alpha=0.75)
plt.title(title)
xtitle=VAR_name+' ('+VAR_unit+' )' 
plt.xlabel(xtitle)
figfilename='Histo_'+VAR_name+'_'+YEARSTR+'_allyear.png'
plt.savefig(figfilename)

# Selection of a period

In [ ]:
df_year['time'].values

In [ ]:
all_datetime=pd.to_datetime(df_year['time'].values)

In [ ]:
all_datetime

In [ ]:
if obs == "lsst":
    summer_time_index=np.where(np.logical_and(all_datetime>dt.datetime(YEARNUM,1,1),all_datetime<dt.datetime(YEARNUM,3,31)))
    winter_time_index=np.where(np.logical_and(all_datetime>dt.datetime(YEARNUM,6,1),all_datetime<dt.datetime(YEARNUM,8,31)))
else:
    winter_time_index=np.where(np.logical_and(all_datetime>dt.datetime(YEARNUM,1,1),all_datetime<dt.datetime(YEARNUM,3,31)))
    summer_time_index=np.where(np.logical_and(all_datetime>dt.datetime(YEARNUM,6,1),all_datetime<dt.datetime(YEARNUM,8,31)))
    

In [ ]:
var_winter=var_year.values[winter_time_index]
var_summer=var_year.values[summer_time_index]

In [ ]:
mean_var_winter = np.mean(var_winter)
median_var_winter = np.median(var_winter)
variance_var_winter = np.var(var_winter)
sigma_var_winter = np.std(var_winter)
sigmamed_var_winter = np.std(var_winter-median_var_winter)

In [ ]:
mean_var_summer = np.mean(var_summer)
median_var_summer = np.median(var_summer)
variance_var_summer = np.var(var_summer)
sigma_var_summer = np.std(var_summer)
sigmamed_var_summer = np.std(var_summer-median_var_summer)

In [ ]:
mw=np.trunc(mean_var_winter*1000)/1000
medw=np.trunc(median_var_winter*1000)/1000
sw=np.trunc(sigma_var_winter*1000)/1000
smedw=np.trunc(sigmamed_var_winter*1000)/1000

ms=np.trunc(mean_var_summer*1000)/1000
ss=np.trunc(sigma_var_summer*1000)/1000
meds=np.trunc(median_var_summer*1000)/1000
smeds=np.trunc(sigmamed_var_summer*1000)/1000

In [ ]:
textstr_s = '\n'.join(("summer:",
    r'$\mu=%.3f$' % (ms, ),
    r'$\mathrm{median}=%.3f$' % (meds, ),
    r'$\sigma_{med}=%.3f$' % (smeds, )))

In [ ]:
textstr_w = '\n'.join(("winter:",
    r'$\mu=%.3f$' % (mw, ),
    r'$\mathrm{median}=%.3f$' % (medw, ),
    r'$\sigma_{med}=%.3f$' % (smedw, )))

In [ ]:
title = "Yearly variation of {}({}) (year {}), summer : {} +/- {}, winter : {} +/- {}".format(VAR_title,VAR_unit,YEARSTR,ms,ss,mw,sw)

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(14,8))
n, bins, patches = ax.hist(var_winter, 50, range=(VAR_min,VAR_max),density=False, facecolor='blue', alpha=0.75,label='winter')
n, bins, patches = ax.hist(var_summer, 50, range=(VAR_min,VAR_max),density=False, facecolor='red', alpha=0.75,label='summer')
ax.set_title(title)
xtitle=VAR_name+' ('+VAR_unit+' )' 
ax.set_xlabel(xtitle)
figfilename='Histo_'+VAR_name+'_'+YEARSTR+'_WinterSummer.png'
ax.legend()

props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords
ax.text(0.5, 0.95, textstr_w, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

ax.text(0.7, 0.95, textstr_s, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

plt.savefig(figfilename)